In [ ]:
! pip install mxnet

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
import pandas as pd
import mxnet as mx
from mxnet import nd, autograd, gluon
from mxnet.gluon import Block, nn, rnn, Trainer
from mxnet.gluon.parameter import Parameter

In [11]:
def relu(X):
    return nd.maximum(X, nd.zeros_like(X))

def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

def logistic(z):
    return 1. / (1. + nd.exp(-z))

def log_loss(output, y):
    yhat = logistic(output)
    return  - nd.nansum(  y * nd.log(yhat) + (1-y) * nd.log(1-yhat))

def cross_entropy(output, y):
    yhat = logistic(output)
    return - nd.nansum(y * nd.log(yhat), axis=0, exclude=True)

In [92]:
mx.random.seed(7)

migrant = [1,0,0,0]
bigot = [0,1,0,0]
love = [0,0,1,0]
hate = [0,0,0,1]

docs = nd.array([
    [love, migrant],
    [hate, migrant] ,
    [love, bigot],
    [hate, bigot]
])

y = nd.array([1,0,0,1])

def xavier(x, i, o):
    return math.sqrt(3./(.5*(i+o)))

W1 = nd.random_normal(shape=(4,2))
b1 = nd.array([0])

W2 = nd.random_normal(shape=(2,2))
b2 = nd.random_normal(shape=2)

W3 = nd.random_normal(shape=2)
b3 = nd.random_normal(shape=1)

params = [W1, b1, W2, b2, W3, b3]

for param in params:
    param.attach_grad()

def net(inputs):
    mem = nd.zeros(shape=2)
    first, second = inputs
    mem = nd.dot(first, W1) + b1
    final = nd.dot(second, W1) + b1
    h1 = relu(mem + final)
    h2 = relu(nd.dot(h1, W2) + b1)
    yhat_linear = nd.dot(h1, W3) + b3
    return yhat_linear

In [93]:
mx.random.seed(1)
from random import shuffle 
dat = list(zip(docs, y))

def train(epochs, learning_rate, loss_fn = log_loss):
    for e in range(epochs):
        cumulative_loss = 0
        # shuffle(dat)
        for i, (data, label) in enumerate(dat):
            with autograd.record():
                output = net(data)
                loss = loss_fn(output, label)
                loss.backward()
            SGD(params, learning_rate)
            cumulative_loss += nd.sum(loss).asscalar()
    print("Loss: %s" % cumulative_loss)

train(1000, .01, log_loss)

Loss: 0.205363826826


In [96]:
for i in range(4):
    print(logistic(net(docs[i])))


[ 0.92139786]
<NDArray 1 @cpu(0)>

[ 0.0211373]
<NDArray 1 @cpu(0)>

[ 0.01947443]
<NDArray 1 @cpu(0)>

[ 0.92139786]
<NDArray 1 @cpu(0)>


In [99]:
params[0]


[[ 1.12406003 -1.47985768]
 [-1.34351015  1.1494199 ]
 [-0.19955443  2.40315962]
 [ 2.26388025 -0.22831535]]
<NDArray 4x2 @cpu(0)>

In [98]:
nd.dot(docs[0], params[0])


[[ 1.12406003 -1.47985768]
 [-0.19955443  2.40315962]]
<NDArray 2x2 @cpu(0)>

In [100]:
nd.dot(docs[1], params[0])


[[-1.34351015  1.1494199 ]
 [-0.19955443  2.40315962]]
<NDArray 2x2 @cpu(0)>

In [101]:
nd.dot(docs[2], params[0])


[[ 1.12406003 -1.47985768]
 [ 2.26388025 -0.22831535]]
<NDArray 2x2 @cpu(0)>

In [103]:
nd.dot(docs[3], params[0])


[[-1.34351015  1.1494199 ]
 [ 2.26388025 -0.22831535]]
<NDArray 2x2 @cpu(0)>